In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd
import pandas as pd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)
from utils.downloads import _prepare_cache

# Porcentagem de pessoas no distrito a menos de 1km de parques urbanos e lineares

Pessoas de outras cidades também estão dentro das áreas de 1km de parques, porém optei por não considerá-las ao fazer a contagem.

In [3]:
data_path = get_data_diretorio()

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook depende do arquivo resultante do notebook 'pop_1km_parques_urb_line'

In [5]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_lineares_e_urbanos',
        'pop_1km_parques_urbanos_lineares.parquet'
    )
)

In [6]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort
20,3550308,São Paulo,355030821,CASA VERDE,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,7.155915,80536,35698,"POLYGON ((331622.857 7399337.293, 331645.702 7...",70699.0
67,3550308,São Paulo,355030868,SACOMA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,14.610797,261436,113894,"POLYGON ((338605.158 7385601.06, 338598.678 73...",0.0


# Calcular a porcentagem da população no recorte

Já temos a quantidade de pessoas por distrito, mas o indicador é: $$ \text{Porcentagem da população a menos de 1Km de parques} = \frac{\text{população total}}{\text{quantidade de pessoas a 1Km ou menos de parques}}$$

In [7]:
gdf_distritos['pct_pop_recort']= (
    gdf_distritos['pop_dist_recort']/gdf_distritos['total_pop']
)

In [8]:
gdf_distritos.sample(3)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
51,3550308,São Paulo,355030852,MARSILAC,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,207.501260,11451,6139,"POLYGON ((333016.008 7354600.919, 333056.044 7...",0.0,0.000000
92,3550308,São Paulo,355030893,VILA PRUDENTE,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.580243,105690,49782,"POLYGON ((339321.368 7389055.527, 339301.644 7...",51258.0,0.484984
94,3550308,São Paulo,355030895,SAO DOMINGOS,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.914590,88884,37746,"POLYGON ((321653.073 7399213.8, 321651.611 739...",66353.0,0.746512


# Conferir

Novamente vamos checar vendo se os mínimos e máximos fazem sentido.

Aqui tem os distritos que não tem nenhuma pessoa a 1km ou menos de parques lineares e urbanos.

In [9]:
(
    gdf_distritos
    .loc[gdf_distritos['pct_pop_recort']==gdf_distritos['pct_pop_recort']
    .min()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
51,3550308,São Paulo,355030852,MARSILAC,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,207.501260,11451,6139,"POLYGON ((333016.008 7354600.919, 333056.044 7...",0.0,0.0
67,3550308,São Paulo,355030868,SACOMA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,14.610797,261436,113894,"POLYGON ((338605.158 7385601.06, 338598.678 73...",0.0,0.0
78,3550308,São Paulo,355030879,SOCORRO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.261579,38051,16549,"POLYGON ((326832.519 7378799.896, 326814.871 7...",0.0,0.0


Aqui é o distrito com a menor porcentagem de pessoas a 1km ou menos de parques lineares e urbanos, menos os que estão zerados

In [10]:
distritos_sem0=gdf_distritos.loc[gdf_distritos['pct_pop_recort']!=0.0]
(
    distritos_sem0
    .loc[distritos_sem0['pct_pop_recort']==distritos_sem0['pct_pop_recort']
    .min()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
26,3550308,São Paulo,355030827,CURSINO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.060052,103171,46155,"POLYGON ((335822.146 7385759.521, 335833.043 7...",131.0,0.00127


Aqui vemos o(s) distrito(s) com a maior porcentagem de pessoas a 1km ou menos dos parques.

In [11]:
(
    gdf_distritos
    .loc[gdf_distritos['pct_pop_recort']==gdf_distritos['pct_pop_recort']
    .max()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort,pct_pop_recort
64,3550308,São Paulo,355030865,RAPOSO TAVARES,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.164313,117738,49006,"POLYGON ((319022.919 7388900.499, 318990.285 7...",117738.0,1.0
93,3550308,São Paulo,355030894,VILA SONIA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,10.066693,123748,53826,"POLYGON ((323710.87 7387720.911, 323703.262 73...",123748.0,1.0


Enquanto todos os outros se mantiveram como a quantidade de pessoas, os distritos com maior porcentagem são diferentes do com maior quantidade. O que provavelmente se explica por Grajaú (maior quantidade) ser um distrito que tem uma população maior de forma geral (o que neste caso, também se reflete no tamanho do distrito):

In [12]:
series_max = {'355030830', '355030865', '355030894'}
gdf_distritos.loc[gdf_distritos['CD_DIST'].isin(series_max)].explore()

# Salvar

In [13]:
save_parquet_excel(
    gdf_distritos,
    'pct_pop_1km_parques_urbanos_lineares',
    assets_path,
    'parques_lineares_e_urbanos'
)